In [1]:
# import tensorflow as tf
# from keras import backend as K

# num_cores = 4
# CPU = True
# GPU = False

# if GPU:
#     num_GPU = 1
#     num_CPU = 1
# if CPU:
#     num_CPU = 1
#     num_GPU = 0

# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
#         inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
#         device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
# session = tf.Session(config=config)
# K.set_session(session)

Using TensorFlow backend.


In [15]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import *
import matplotlib.pyplot as plt
from tokenizer import tokenizer as tweet_tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import * 
from keras.regularizers import *
from keras.models import load_model
import pickle
import json


%matplotlib inline

PATH = './data'

path_train_3 = f'{PATH}/data_train_3.csv'
path_test_3 = f'{PATH}/data_test_3.csv'
path_train_7 = f'{PATH}/data_train_7.csv'
path_train_16m_3 = f'{PATH}/training.1600000.processed.noemoticon.csv'

# Serious business

## Preprocess Train_3

In [4]:
tweets = pd.read_csv(path_train_3, sep='\t', names=['ID', 'Class', 'Tweet'])
tweets_7 = pd.read_csv(path_train_7, sep='\t', names=['ID', 'Class', 'Tweet'], dtype={'Tweet': str})
tweets_3_test = pd.read_csv(path_test_3, sep='\t', names=['ID', 'Class', 'Tweet'], dtype={'Tweet': str})

tweets.shape, tweets_7.shape, tweets_3_test.shape

((50333, 3), (1630, 3), (1630, 3))

In [ ]:
tweets.sample(5)

In [31]:
tweets_7.head(5)

,ID,Class,Tweet
0,0,0,@liamch88 yeah! :) playing well
1,1,0,At least I don't have a guy trying to discoura...
2,2,0,UPLIFT: If you're still discouraged it means y...
3,3,0,"...at your age, the heyday in the blood is tam..."
4,4,-2,i was so embarrassed when she saw us i was lik...


In [20]:
from TP_transfer_learning_2018 import *
from TP_transfer_learning_2018.preprocessing import standardization


tweets['Sentiment'] = tweets['Class'].apply(lambda x: {'negative': 0, 'neutral': 1, 'positive': 2}[x])
tweets['Tweet'] = tweets['Tweet'].apply(lambda x: standardization(x))

In [32]:
tweets_7['Sentiment'] = tweets_7['Class']
tweets_7['Tweet'] = tweets_7['Tweet'].apply(lambda x: standardization(x))

In [33]:
tweets_3_test['Sentiment'] = tweets_3_test['Class'].apply(lambda x: {'negative': 0, 'neutral': 1, 'positive': 2}[x])
tweets_3_test['Tweet'] = tweets_3_test['Tweet'].apply(lambda x: standardization(x))

In [34]:
display(tweets.sample(2))
display(tweets_7.head(2))
display(tweets_3_test.head(2))

,ID,Class,Tweet
28257,635721210806956032,neutral,For an art project in 7th grade we had to make...
17615,634516378268119041,negative,.@marklevinshow apparently Rick Perry is an id...


,ID,Class,Tweet,Sentiment
0,0,0,yeah ☺ ️ playing well,0
1,1,0,least not guy try discourage anymore want neve...,0


,ID,Class,Tweet,Sentiment
0,449,negative,site crash everytime try book help tell nothin...,0
1,450,negative,theme week ask lord strength perspective perse...,0


In [35]:
tweets_7.to_feather('pickles/train_7.feather')

In [36]:
tweets_3_test.to_feather('pickles/test_3.feather')

In [22]:
tweets.to_feather('pickles/train_3.feather')

## Training

In [5]:
tweets = pd.read_feather('pickles/train_3.feather')
test_3 = pd.read_feather('pickles/test_3.feather')
train_7 = pd.read_feather('pickles/train_7.feather')

train_x, train_y = tweets['Tweet'], tweets['Sentiment']
test_x, test_y = test_3['Tweet'], test_3['Sentiment']
train7_x, train7_y = train_7['Tweet'], train_7['Sentiment']

* Fitting the tokenizer on our entire tweet base to make sure the indexes are rights

In [6]:
all_tweets = pd.concat([train_x, test_x, train7_x])
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(all_tweets)
word_index = tokenizer.word_index

pickle.dump(tokenizer, open("pickles/tokenizer.pickle", "wb"))

In [7]:
tokenizer = pickle.load(open("pickles/tokenizer.pickle", 'rb'))

* Looking for the MAX_SEQUENCE_LENGTH in all our sequences:

In [8]:
train_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences = tokenizer.texts_to_sequences(test_x)
train7_sequences = tokenizer.texts_to_sequences(train7_x)

sequences = train_sequences + test_sequences + train7_sequences
MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

MAX_SEQUENCE_LENGTH

32

* We only train on train_3:

In [9]:
train_sequences = pad_sequences(train_sequences, MAX_SEQUENCE_LENGTH)
train_sequences.shape

(50333, 32)

#### Loading Datastories's embeddings

In [10]:
# googlenews_w2v = KeyedVectors.load_word2vec_format('data/embeddings/GoogleNews-vectors-negative300.bin', binary=True)
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import get_tmpfile


tmp_file = get_tmpfile('datastories.300d.word2vec')
glove2word2vec('data/embeddings/datastories.twitter.300d.txt', tmp_file)
w2v = KeyedVectors.load_word2vec_format(tmp_file)

### EmoLex

In [12]:
emolex = pd.read_csv('TP_transfer_learning_2018/EmoLex.txt', sep='\t')

### OLE

In [11]:
positive_words = []
negative_words = []

with open('data/positive-words.txt') as positive_file, open('data/negative-words.txt', encoding='ISO-8859-1') as negative_file:
    for _ in range(35):
        next(positive_file)
        next(negative_file)
        
    for line in positive_file:
        positive_words.append(line)
    for line in negative_file:
        negative_words.append(line)

### Emoji valence and AFINN

In [13]:
afinn = pd.read_csv('data/AFINN-111.txt', sep='\t')

def val_to_list(x):
    x += 5
    return(to_categorical(x, num_classes=11, dtype='int'))


afinn[['val']] = afinn['val'].apply(val_to_list)
print(afinn.head(10))
print(afinn.loc[afinn['word'] == 'abandon'].val)

         word                                val
0     abandon  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
1   abandoned  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2    abandons  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
3    abducted  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
4   abduction  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
5  abductions  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
6       abhor  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
7    abhorred  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
8   abhorrent  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
9      abhors  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Name: val, dtype: object


In [16]:
with open('./data/index.json') as emojiFile:
    emoji_valence = json.load(emojiFile)

for elmt in emoji_valence:
    val = elmt['polarity']
    elmt['polarity'] = val_to_list(val)
emoji_valence

[{'emoji': '💯',
  'name': '100',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '😠',
  'name': 'angry',
  'polarity': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'emoji': '😧',
  'name': 'anguished',
  'polarity': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'emoji': '😲',
  'name': 'astonished',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])},
 {'emoji': '🖤',
  'name': 'black_heart',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '💙',
  'name': 'blue_heart',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '😊',
  'name': 'blush',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])},
 {'emoji': '💔',
  'name': 'broken_heart',
  'polarity': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])},
 {'emoji': '👏',
  'name': 'clap',
  'polarity': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])},
 {'emoji': '🤡',
  'name': 'clown_face',
  'polarity': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])},
 {'emoji': '😰',
  'name': 'cold_sweat',
  'polarity

### Depeche Mood

In [17]:
DepecheMoodpp = pd.read_csv('data/DepecheMood/DepecheMood_english_token_full.tsv', sep='\t')
DepecheMood = pd.read_csv('data/DepecheMood/DepecheMood_freq.txt', sep='\t')

print(DepecheMood.columns.values)
print(DepecheMoodpp.columns.values)

['Lemma#PoS' 'AFRAID' 'AMUSED' 'ANGRY' 'ANNOYED' 'DONT_CARE' 'HAPPY'
 'INSPIRED' 'SAD']
['word' 'AFRAID' 'AMUSED' 'ANGRY' 'ANNOYED' 'DONT_CARE' 'HAPPY' 'INSPIRED'
 'SAD' 'freq']


### Embedding matrix

In [ ]:
targets = to_categorical(train_y, 3)
nb_words = len(word_index) + 1

EMBEDDING_DIM = 300
EMOLEX_DIM = 10
OLE_DIM = 2
EMOJI_VALENCE_DIM = 11
AFINN_DIM = 11 
DEPECHE_MOOD_DIM = 8
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM + EMOLEX_DIM + OLE_DIM + AFINN_DIM + DEPECHE_MOOD_DIM + EMOJI_VALENCE_DIM))

oov = []  # Out of vocabulary
oov.append((np.random.rand(EMBEDDING_DIM) * 2.0) - 1.0)
oov = oov / np.linalg.norm(oov)
empty_afinn = np.full(11, 0)
empty_emoji = np.full(11, 0)
empty_emolex = np.full(10, 0)
empty_depeche = np.full(8, 0)


print(empty_emolex)
print(oov.shape)

for word, i in word_index.items():
    emoji_val = 0
    
    word_vector = oov
    if word in w2v.vocab:
        word_vector = w2v.word_vec(word)

    emolex_row = emolex.loc[emolex['word'] == word]
    if emolex_row.empty:
        word_vector = np.append(word_vector, empty_emolex)
    else:
        word_vector = np.append(word_vector, emolex_row.values.tolist()[0][1:])
        
    depeche_row = DepecheMoodpp.loc[DepecheMoodpp['word'] == word]
    if depeche_row.empty:
        word_vector = np.append(word_vector, empty_depeche)
    else:
        word_vector = np.append(word_vector, depeche_row.values.tolist()[0][1:9])
        
    ole_val = [0, 0]
    if word in positive_words:
        ole_val = [1, 0]
    elif word in negative_words:
        ole_val = [0, 1]
    word_vector = np.append(word_vector, ole_val)
    
    afinn_val = empty_afinn
    emoji_val = empty_emoji
    
    afinn_row = afinn.loc[afinn['word'] == word]
    if not afinn_row.empty:
        afinn_val = afinn_row['val'].item()
    else:
        for emoji in emoji_valence:
            if word == emoji['emoji']:
                emoji_val = emoji['polarity']

    word_vector = np.append(word_vector, afinn_val)
    
    word_vector = np.append(word_vector, emoji_val)
    
    embedding_matrix[i] = word_vector

        
print(embedding_matrix.shape)

[0 0 0 0 0 0 0 0 0 0]
(1, 300)


In [90]:
# Claim memory back from this very large object we don't use anymore
del w2v

In [91]:
X_train, X_val, y_train, y_val = train_test_split(train_sequences, targets, test_size=0.3)

print('training set: ' + str(len(X_train)) + ' samples')
print('validation set: ' + str(len(X_val)) + ' samples')

print('x_train:', X_train.shape)
print('y_train:', y_train.shape)

training set: 35233 samples
validation set: 15100 samples
x_train: (35233, 32)
y_train: (35233, 3)


In [92]:
def model_2017():
    vocab_size = embedding_matrix.shape[0]
    embedding_size = embedding_matrix.shape[1]

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False, name='embedding_layer'))
    model.add(GaussianNoise(0.3))
    # model.add(SpatialDropout1D(0.3))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, recurrent_dropout=0.3, return_sequences=True)))
    # model.add(Dropout(0.5))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, recurrent_dropout=0.3, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(3, activity_regularizer=l2(1e-4)))
    model.add(Activation('softmax'))

    # RMSprop or lr=1e-3
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-2), metrics=['acc'])

    print(model.summary())
    
    return model

def model_mine():
    vocab_size = embedding_matrix.shape[0]
    embedding_size = embedding_matrix.shape[1]

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights=[embedding_matrix], 
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False, name='embedding_layer'))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    print(model.summary())
    
    return model

In [93]:
model = model_mine()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11335500  
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
__________

In [94]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=5)

Train on 35233 samples, validate on 15100 samples
Epoch 1/5
35233/35233 [==============================] - 129s 4ms/step - loss: 0.8453 - acc: 0.5889 - val_loss: 0.7882 - val_acc: 0.6365
Epoch 2/5
35233/35233 [==============================] - 124s 4ms/step - loss: 0.7753 - acc: 0.6378 - val_loss: 0.7691 - val_acc: 0.6319
Epoch 3/5
35233/35233 [==============================] - 127s 4ms/step - loss: 0.7457 - acc: 0.6527 - val_loss: 0.7332 - val_acc: 0.6591
Epoch 4/5
35233/35233 [==============================] - 126s 4ms/step - loss: 0.7211 - acc: 0.6665 - val_loss: 0.7522 - val_acc: 0.6555
Epoch 5/5
35233/35233 [==============================] - 126s 4ms/step - loss: 0.7030 - acc: 0.6776 - val_loss: 0.7273 - val_acc: 0.6644


In [99]:
test_sequences = pad_sequences(test_sequences, MAX_SEQUENCE_LENGTH)
test_sequences.shape
Y_test = to_categorical(test_y, 3)

In [101]:
loss, acc = model.evaluate(test_sequences, Y_test, batch_size=128)

print(f'val_loss: {loss}\nval_acc: {acc}')

1630/1630 [==============================] - 2s 1ms/step


[1.0056506370474225, 0.5263803679518905]

In [104]:
model.save('pickles/train_3_65-mine.h5')